In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [17]:
hospitals = pd.read_csv('hospitals_0.csv')
hospitals.drop(columns=['PPN / NON PPN', 'Address'], inplace=True)

In [19]:
hospitals['Rating'] = [None] * len(hospitals)
hospitals['Number of Reviews'] = [None] * len(hospitals)
hospitals['Plus Code'] = [None] * len(hospitals)

In [20]:
def getFullAddress(rowIndex):
    print("Generating full address for row", rowIndex)
    return ', '.join(hospitals.iloc[rowIndex].dropna()).replace('\n', '').replace('\r', '')

In [21]:
def searchForAddress(address, driver):
    print("Searching for address:", address)
    search_box = driver.find_element(By.ID, 'searchboxinput')
    search_box.clear() 
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)         

    print("Searched")

In [22]:
"""
EXAMPLE HTML
<div class="F7nice " jslog="76333;mutable:true;">
    <span>
        <span aria-hidden="true">4.8</span>
        <span class="ceNzKf" role="img" aria-label="4.8 stars ">
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
            <span class="rFrJzc"></span>
        </span>
    </span>
    <span>
        <span>
            <span aria-label="1,168 reviews">(1,168)</span>
        </span>
    </span>
</div>
"""

def scrape_data(driver):
    time.sleep(2)
    print("Scraping data")
    if driver.current_url.startswith("https://www.google.com/maps/search/"):
        print("In search page")
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.hfpxzc"))
            ).click()
        except Exception as e:
            print("No search results found:", e)
            return None, None, None


    def get_element_text(selector, attribute=None):
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            return element.get_attribute(attribute) if attribute else element.text
        except Exception as e:
            print(f"Error scraping {selector}:", e)
            return None

    rating = get_element_text("div.F7nice span[aria-hidden='true']")
    num_reviews = get_element_text("div.F7nice span > span > span[aria-label]", "aria-label")
    if num_reviews:
        num_reviews = int(num_reviews.strip('()').replace(',', '').split(' ')[0])
    plus_code = get_element_text(".AeaXub .Io6YTe")

    print("Scraped data:", rating, num_reviews, plus_code)
    return rating, num_reviews, plus_code




In [23]:
edge_driver_path = "E:/Downloads/edgedriver_win64/msedgedriver.exe"
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

driver.get('https://www.google.com/maps')
with open("error_log.tc", "w") as f:
    for i in range(2, len(hospitals)):
        # try:
            print(f"Processing row {i}")
            address = getFullAddress(i)
            searchForAddress(address, driver)


            rating, num_reviews, plus_code = scrape_data(driver)

            hospitals.at[i, 'Rating'] = rating
            hospitals.at[i, 'Number of Reviews'] = num_reviews
            hospitals.at[i, 'Plus Code'] = plus_code

            close_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Close']"))
            )
            close_button.click()
            time.sleep(2)  

            if i % 10 == 0:
                hospitals.to_csv('hospitals_0_ratings.csv', index=False)

        # except Exception as e:
            # f.write(f"Error at index {i}: {str(e)}\n")
            # f.flush()
            # continue
        
# driver.quit()

Processing row 2
Generating full address for row 2
Searching for address: Aashraya Multi Specialty Hospital, CHITOOR, ANDHRA PRADESH
Searched
Scraping data
In search page
Scraped data: 3.9 139 #10/259, behind Vinayaka temple, Kattamanchi, Chittoor, Andhra Pradesh 517001
Processing row 3
Generating full address for row 3
Searching for address: Aragonda Apollo Hospitals, Chittoor, ANDHRA PRADESH
Searched
Scraping data
In search page
Scraped data: 4.0 92 Chittoor Aragonda Rd, Aragonda, Andhra Pradesh 517129
Processing row 4
Generating full address for row 4
Searching for address: Apollo Samudra Hospitals(Kakinada), East Godavari, ANDHRA PRADESH
Searched
Scraping data
Scraped data: 4.8 8090 13-1-3, Main Rd, Surya Rao Peta, Kakinada, Andhra Pradesh 533001
Processing row 5
Generating full address for row 5
Searching for address: Swatantra Hospitals Pvt Ltd, East Godavari, ANDHRA PRADESH
Searched
Scraping data
Scraped data: 4.4 467 Kambala Park Rd, opposite to Income Tax Office, Kambala Cheru

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=130.0.2849.56)
Stacktrace:
	GetHandleVerifier [0x00007FF66594DC75+12853]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665BF83C4+2250276]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665B376A6+1460486]
	(No symbol) [0x00007FF6656E84C7]
	(No symbol) [0x00007FF66576CBED]
	(No symbol) [0x00007FF665780D7A]
	(No symbol) [0x00007FF665768163]
	(No symbol) [0x00007FF66573FB54]
	(No symbol) [0x00007FF66573F000]
	(No symbol) [0x00007FF66573F741]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665ACCDB4+1024020]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF665869A04+57108]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665ACBDB3+1019923]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF665ACBA19+1019001]
	Microsoft::Applications::Event